In [ ]:
import os
import pickle
from pathlib import Path
import yaml

import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage

from dasha.strategy import _CompressionAggregator

RESULTS = Path(os.path.abspath("")).parent / "results"
OUTPUT_PATH = Path(os.path.abspath("")).parent / "_static"
OUTPUT_PATH.mkdir(exist_ok=True, parents=True)

In [ ]:
def moving_average(x, window):
    """Smooth plot using a window."""
    window = np.ones((window,)) / window
    return scipy.ndimage.convolve1d(x, window)


def plot(input_paths:list, metric_name:str, output_path: str, smooth_plot:int = 0) -> None:
    """Plot experiments."""
    fig, axs = plt.subplots(nrows=1, ncols=1, sharex="row")
    paths = []
    for path in input_paths:
        if not os.path.exists(os.path.join(RESULTS, path, "history.pkl")):
            local_folders = os.listdir(path)
            for folder in local_folders:
                if os.path.isdir(os.path.join(path, folder)):
                    paths.append(os.path.join(path, folder))
        else:
            paths.append(path)
    for save_path in paths:
        with open(os.path.join(RESULTS, save_path, "history.pkl"), "rb") as file:
            history = pickle.load(file)["history"]
        with open(os.path.join(RESULTS, save_path, "config.json"), "r") as file:
            cfg = yaml.safe_load(file)
        metric = history.metrics_distributed[_CompressionAggregator.RECEIVED_BYTES]
        received_bytes_rounds, received_bytes = list(zip(*metric))
        if metric_name == "loss":
            rounds, losses = list(zip(*history.losses_distributed))
            axs.set_yscale("log")
        elif metric_name == _CompressionAggregator.SQUARED_GRADIENT_NORM:
            metrics = history.metrics_distributed[
                _CompressionAggregator.SQUARED_GRADIENT_NORM
            ]
            rounds, losses = list(zip(*metrics))
            axs.set_yscale("log")
        elif metric_name == _CompressionAggregator.ACCURACY:
            metrics = history.metrics_distributed[_CompressionAggregator.ACCURACY]
            rounds, losses = list(zip(*metrics))
        np.testing.assert_array_equal(received_bytes_rounds, rounds)
        target = cfg["method"]["name"]
        client = target.split(".")[-1]
        losses = np.asarray(losses)
        if smooth_plot:
            losses = moving_average(losses, smooth_plot)
        step_size = cfg["method"]["step-size"]
        axs.plot(
            np.asarray(received_bytes),
            losses,
            label=f"{client}; Step size: {step_size}",
        )
        axs.set_ylabel(metric_name)
        axs.set_xlabel("#bits / client")
        axs.grid(True)
        axs.legend(loc="upper left")
        fig.savefig(output_path)

# Small-Scale Experiments

In [ ]:
experiments_list = [
    "mushrooms/dasha/0.25/",
    "mushrooms/dasha/0.5/",
    "mushrooms/dasha/1.0/",
    "mushrooms/marina/0.25/",
    "mushrooms/marina/0.5/",
    "mushrooms/marina/1.0/",
]
plot(experiments_list, metric_name="squared_gradient_norm", output_path= OUTPUT_PATH / "plot.png")

# Large-Scale Experiment

In [ ]:
experiments_list = [
    "cifar10/stochastic_dasha/0.01/",
    "cifar10/stochastic_marina/0.01/",
]
plot(experiments_list, metric_name="squared_gradient_norm", output_path= OUTPUT_PATH / "plot_nn.png", smooth_plot=100)